In [1]:
# Import packages:

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [2]:
# Scrape raw text from the link:

link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
txt = requests.get(link).text
raw = BeautifulSoup(txt, 'lxml')
content = raw.find('div', class_='mw-parser-output')
table = content.table.tbody

In [3]:
# Find relevant information and construct table:

trs = table.find_all('tr')
tds = []
for tr in trs:
    tds.append(tr.find_all('td'))
postcodes = []
boroughs = []
neighborhoods = []
for td in tds[1:]:
    postcodes.append(td[0].text)
    boroughs.append(td[1].text)
    neighborhoods.append(td[2].text.strip('\n'))
df = pd.DataFrame({'PostalCode': postcodes, 'Borough': boroughs, 'Neighborhood': neighborhoods})
df_valid = df[(df['Borough'] != 'Not assigned')].reset_index(drop = True)
df_valid['Neighborhood'] = np.where(df_valid['Neighborhood'] == 'Not assigned', df_valid['Borough'], \
                                    df_valid['Neighborhood'])
df_grouped = df_valid.groupby(by = ['PostalCode', 'Borough'])['Neighborhood'].apply(lambda x: ','.join(x))\
                     .reset_index()
df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [4]:
# Dimensions:

df_grouped.shape

(103, 3)

In [5]:
geo = pd.read_csv('Geospatial_Coordinates.csv').rename(columns = {'Postal Code': 'PostalCode'})
df_final = df_grouped.merge(geo, how = 'left', on = 'PostalCode')\
                     .rename(columns = {'Neighborhood': 'Neighborhood Name'})
df_final.head()

,PostalCode,Borough,Neighborhood Name,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [6]:
# Dimensions:

df_final.shape

(103, 5)

With the data ready, we can analyze which neighborhoods are similar. This is measured the similarity in the venues around the neighborhood. I will use Foursquare API to find the top 100 venues of each neighborhood.


The first step is to define Foursquare API credentials.

In [10]:
# Define Foursquare Credentials and Version

CLIENT_ID = 'EMSLPV50G0YSZ2XXIYHSIY1O1ETFRNPEGE35IPESQVQ0KK2B' # your Foursquare ID
CLIENT_SECRET = 'RT0F4XIVWEHDE15GN52KCUTEZIYRZ30DAUJ1O5OQ30B13BHR' # your Foursquare Secret
VERSION = '20180605' 
LIMIT = 100
radius = 500

With parameters set, define the function to get venues nearby each neighborhood:

In [15]:
# Define function to get venues:

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name'],
            v['venue']['id']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood Name', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category',
                  'Venue ID']
    
    return(nearby_venues)

In [16]:
# Get venues within 500 radius of Toronto neighborhoods. Drop venue ID column for now.

toronto_venues = getNearbyVenues(names = df_final['Neighborhood Name'],\
                                 latitudes = df_final['Latitude'],
                                 longitudes = df_final['Longitude']).drop(columns = ['Venue ID'])

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

In [17]:
# First five records of Toronto venues:

toronto_venues.head()

,Neighborhood Name,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge,Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place


Then for the same neighborhood, get dummies for each venue category

In [18]:
# Get dummies for each venue category:
toronto_dummy = pd.get_dummies(toronto_venues[['Venue Category']], prefix = '', prefix_sep = '')

# Add neighborhood name and move to first column:
toronto_dummy.insert(0,'Neighborhood Name', toronto_venues['Neighborhood Name'])

Then for each neighborhood, find the average number of venues:

In [19]:
# Group venues for same neighborhoods and return the mean:
toronto_mean = toronto_dummy.groupby('Neighborhood Name').mean().reset_index()

Then cluster neithbothoods using k-means:

In [20]:
# run k-means clustering with k = 5:
from sklearn.cluster import KMeans
toronto_mean_clustering = toronto_mean.drop(columns = ['Neighborhood Name'])
kmeans = KMeans(n_clusters = 5, random_state=0).fit(toronto_mean_clustering)

# Add neighborhood and latitude, longtitude information
toronto_mean.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_cluster = toronto_mean.merge(df_final, how = 'left', on = ['Neighborhood Name'])


In [21]:
# First 5 records of clustereed neithborhoods:

toronto_cluster.head()

,Cluster Labels,Neighborhood Name,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio,PostalCode,Borough,Latitude,Longitude
0,0,"Adelaide,King,Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.01,0.0,0.0,0.0,M5H,Downtown Toronto,43.650571,-79.384568
1,2,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,M1S,Scarborough,43.794200,-79.262029
2,3,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,M1V,Scarborough,43.815252,-79.284577
3,2,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,M9V,Etobicoke,43.739416,-79.588437
4,2,"Alderwood,Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,M8W,Etobicoke,43.602414,-79.543484


Finally, cerate a map to show the clusters:

In [14]:
# pip install folium

In [22]:
# create map

import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
latitude = 43.6532
longitude =  -79.3832
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
kclusters = 5
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_cluster['Latitude'], toronto_cluster['Longitude'], \
                                  toronto_cluster['Neighborhood Name'], toronto_cluster['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters